In [1]:
import os
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import random
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch import nn
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
from peft import get_peft_model, PrefixTuningConfig, TaskType, LoraConfig
from utils import *

In [2]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda:0")

In [3]:
dataset_name = "sst2" #sst2, qnli qqp mnli
tokenized_data = load_cleaned_data(dataset_name)
train_dataloader = DataLoader(tokenized_data['train'], shuffle=False, batch_size=1024,collate_fn=collate_fn)
if dataset_name == "mnli":
    tokenized_data['validation'] = tokenized_data["validation_matched"]
val_dataloader = DataLoader(tokenized_data['validation'], shuffle=False, batch_size=1024,collate_fn=collate_fn)
num_labels = torch.unique(tokenized_data["train"]["labels"]).numel()
loss_fn = nn.CrossEntropyLoss()

Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Tue Mar 25 23:46:02 2025) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.
03/31/2025 23:43:16:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Tue Mar 25 23:46:02 2025) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.


# Full-tuning without DP

In [4]:
model_name = "prajjwal1/bert-tiny"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

optimizer = AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
epochs = 5

model.to(device)

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * epochs),)

trainable_params = count_trainable_params(model)
print(f"The number of trainable parameters: {trainable_params}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The number of trainable parameters: 4386178


In [5]:
trainModel(model,optimizer,train_dataloader,val_dataloader,loss_fn,lr_scheduler,tqdm,dataset_name)

100%|██████████| 66/66 [00:11<00:00,  5.63it/s]
/media/nvidia/00ad733b-135d-4406-9b4f-61c0260a5b45/project/SWX/tiny-bert/utils.py:72: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", task)


epoch=0: train_ppl=tensor(1.4546, device='cuda:0') train_epoch_loss=tensor(0.3747, device='cuda:0') 


Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.
03/31/2025 23:43:33:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.


Epoch 1, Validation Accuracy without DP: {'accuracy': 0.8130733944954128}


100%|██████████| 66/66 [00:10<00:00,  6.44it/s]


...

Epoch 4, Validation Accuracy without DP: {'accuracy': 0.7935779816513762}


100%|██████████| 66/66 [00:08<00:00,  7.97it/s]


epoch=4: train_ppl=tensor(1.1026, device='cuda:0') train_epoch_loss=tensor(0.0976, device='cuda:0') 


Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.
03/31/2025 23:44:26:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.


Epoch 5, Validation Accuracy without DP: {'accuracy': 0.8004587155963303}
Training complete!


# Full-tuning with DP

In [6]:
model_name = "prajjwal1/bert-tiny"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

optimizer = AdamW(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
epochs = 5

model.to(device)

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * epochs),)

trainable_params = count_trainable_params(model)
print(f"The number of trainable parameters: {trainable_params}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The number of trainable parameters: 4386178


In [7]:
dp_train_2(model,train_dataloader,tokenized_data,optimizer,lr_scheduler,epochs,val_dataloader,dataset_name)

  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 66/66 [00:13<00:00,  5.05it/s]
Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.
03/31/2025 23:44:43:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/gl

Epoch 1, Validation Accuracy DP: {'accuracy': 0.8256880733944955}


  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 66/66 [00:12<00:00,  5.15it/s]
Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.
03/31/2025 23:45:00:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/gl

Epoch 2, Validation Accuracy DP: {'accuracy': 0.8268348623853211}


  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 66/66 [00:12<00:00,  5.11it/s]
Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.
03/31/2025 23:45:16:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/gl

Epoch 3, Validation Accuracy DP: {'accuracy': 0.8153669724770642}


  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 66/66 [00:12<00:00,  5.12it/s]
Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.
03/31/2025 23:45:33:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/gl

Epoch 4, Validation Accuracy DP: {'accuracy': 0.8256880733944955}


  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
100%|██████████| 66/66 [00:12<00:00,  5.16it/s]
Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.
03/31/2025 23:45:49:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/gl

Epoch 5, Validation Accuracy DP: {'accuracy': 0.8245412844036697}
Training complete
